In [30]:
import pandas as pd
from sklearn.metrics import cohen_kappa_score, accuracy_score

merged_df = pd.read_csv('spojeni_rezultati.csv')

# Prikaz prvih nekoliko redova
print(merged_df.head()) 


       Ime fajla Aminokiselina  Pozicija ISUNSTRUCT_Klasifikacija  \
0  seq_001.fasta             M         1                neuređena   
1  seq_001.fasta             D         2                neuređena   
2  seq_001.fasta             L         3                neuređena   
3  seq_001.fasta             H         4                neuređena   
4  seq_001.fasta             S         5                neuređena   

  IUPRED3_Klasifikacija VSL2B_Klasifikacija  
0               uređena           neuređena  
1               uređena           neuređena  
2               uređena           neuređena  
3               uređena           neuređena  
4               uređena           neuređena  


/tmp/ipykernel_48865/2154740642.py:4: DtypeWarning: Columns (3) have mixed types. Specify dtype option on import or set low_memory=False.
  merged_df = pd.read_csv('spojeni_rezultati.csv')


In [31]:
# pretvaranje u numericke vrednosti(0 = neuređena, 1 = uređena)
classification_map = {'neuređena': 0, 'uređena': 1}

# upotreba .replace() za direktnu zamenu tekstualnih vrednosti u numeričke
merged_df['ISUNSTRUCT_Klasifikacija'] = merged_df['ISUNSTRUCT_Klasifikacija'].replace(classification_map)
merged_df['IUPRED3_Klasifikacija'] = merged_df['IUPRED3_Klasifikacija'].replace(classification_map)
merged_df['VSL2B_Klasifikacija'] = merged_df['VSL2B_Klasifikacija'].replace(classification_map)

# Prikaz prvih nekoliko redova da proverimo rezultat
print(merged_df.head())


       Ime fajla Aminokiselina  Pozicija  ISUNSTRUCT_Klasifikacija  \
0  seq_001.fasta             M         1                       0.0   
1  seq_001.fasta             D         2                       0.0   
2  seq_001.fasta             L         3                       0.0   
3  seq_001.fasta             H         4                       0.0   
4  seq_001.fasta             S         5                       0.0   

   IUPRED3_Klasifikacija  VSL2B_Klasifikacija  
0                      1                    0  
1                      1                    0  
2                      1                    0  
3                      1                    0  
4                      1                    0  


In [32]:
print(merged_df['ISUNSTRUCT_Klasifikacija'].isna().sum())  # Broj NaN vrednosti u ISUNSTRUCT_numeric
print(merged_df['IUPRED3_Klasifikacija'].isna().sum())  # Broj NaN vrednosti u IUPRED3_numeric
print(merged_df['VSL2B_Klasifikacija'].isna().sum())  # Broj NaN vrednosti u VSL2B_numeric


438778
0
0


Posto moj csv fajl ima NaN vrednosti ja zelim da uklonim redove koji sadrze te NaN vrednosti

In [33]:
merged_cleaned_df = merged_df.copy()
merged_cleaned_df = merged_df.dropna()

merged_cleaned_df.to_csv('spojeni_rezultati_cisti.csv', index=False)

print("Očišćeni DataFrame bez NaN vrednosti:")
print(merged_cleaned_df.head())

Očišćeni DataFrame bez NaN vrednosti:
       Ime fajla Aminokiselina  Pozicija  ISUNSTRUCT_Klasifikacija  \
0  seq_001.fasta             M         1                       0.0   
1  seq_001.fasta             D         2                       0.0   
2  seq_001.fasta             L         3                       0.0   
3  seq_001.fasta             H         4                       0.0   
4  seq_001.fasta             S         5                       0.0   

   IUPRED3_Klasifikacija  VSL2B_Klasifikacija  
0                      1                    0  
1                      1                    0  
2                      1                    0  
3                      1                    0  
4                      1                    0  


Provera konzistentnosti klasifikacija

In [34]:
# dodajemo novu kolonu koja proverava konzistentnost
merged_cleaned_df.loc[:, 'Konzistentno'] = (
    (merged_cleaned_df['ISUNSTRUCT_Klasifikacija'] == merged_cleaned_df['IUPRED3_Klasifikacija']) &
    (merged_cleaned_df['ISUNSTRUCT_Klasifikacija'] == merged_cleaned_df['VSL2B_Klasifikacija'])
)

# racunanje procenta konzistentnosti
konzistentnost_procenat = merged_cleaned_df['Konzistentno'].mean() * 100
print(f"Procenat konzistentnosti između alata: {konzistentnost_procenat:.2f}%")


Procenat konzistentnosti između alata: 83.03%


/tmp/ipykernel_48865/215445941.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  merged_cleaned_df.loc[:, 'Konzistentno'] = (


Ovo znaci da su za 83.03% redova (pozicija u proteinima) svi alati dali istu klasifikaciju, odnosno svi su se slozili da je pozicija uredjena ili neuredjena.

Racunanje Kappa koeficijenta izmedju alata

In [35]:
# Izračunavanje Kappa koeficijenta između ISUNSTRUCT i IUPRED3
kappa_isunstruct_iupred3 = cohen_kappa_score(merged_cleaned_df['ISUNSTRUCT_Klasifikacija'], merged_cleaned_df['IUPRED3_Klasifikacija'])
print(f"Cohen's Kappa između ISUNSTRUCT i IUPRED3: {kappa_isunstruct_iupred3:.2f}")

# Izračunavanje Kappa koeficijenta između ISUNSTRUCT i VSL2B
kappa_isunstruct_vsl2b = cohen_kappa_score(merged_cleaned_df['ISUNSTRUCT_Klasifikacija'], merged_cleaned_df['VSL2B_Klasifikacija'])
print(f"Cohen's Kappa između ISUNSTRUCT i VSL2B: {kappa_isunstruct_vsl2b:.2f}")

# Izračunavanje Kappa koeficijenta između IUPRED3 i VSL2B
kappa_iupred3_vsl2b = cohen_kappa_score(merged_cleaned_df['IUPRED3_Klasifikacija'], merged_cleaned_df['VSL2B_Klasifikacija'])
print(f"Cohen's Kappa između IUPRED3 i VSL2B: {kappa_iupred3_vsl2b:.2f}")


Cohen's Kappa između ISUNSTRUCT i IUPRED3: 0.70
Cohen's Kappa između ISUNSTRUCT i VSL2B: 0.74
Cohen's Kappa između IUPRED3 i VSL2B: 0.59


Cohen's Kappa (cohen_kappa_score) - Ovaj koeficijent meri slaganje između dve klasifikacije, korigujuci slucajno slaganje. Vrednost može biti između -1 i 1:

1 znači savrseno slaganje
0 znači da je slaganje slucajno
Negativne vrednosti znace da je slaganje losije od slucajnog

Nasi rezultati - Zakljucak:
Alati ISUNSTRUCT i IUPRED3 imaju dobro slaganje, dok ISUNSTRUCT i VSL2B imaju vrlo dobro slaganje.
IUPRED3 i VSL2B imaju umereno slaganje, što moze ukazivati da postoji manja konzistentnost između tih alata u poređenju sa prva dva para


In [36]:
from sklearn.metrics import precision_score, accuracy_score

# ISUNSTRUCT vs IUPRED3
accuracy_isunstruct_iupred3 = accuracy_score(merged_cleaned_df['ISUNSTRUCT_Klasifikacija'], merged_cleaned_df['IUPRED3_Klasifikacija'])
precision_isunstruct_iupred3 = precision_score(merged_cleaned_df['ISUNSTRUCT_Klasifikacija'], merged_cleaned_df['IUPRED3_Klasifikacija'])

# ISUNSTRUCT vs VSL2B
accuracy_isunstruct_vsl2b = accuracy_score(merged_cleaned_df['ISUNSTRUCT_Klasifikacija'], merged_cleaned_df['VSL2B_Klasifikacija'])
precision_isunstruct_vsl2b = precision_score(merged_cleaned_df['ISUNSTRUCT_Klasifikacija'], merged_cleaned_df['VSL2B_Klasifikacija'])

# IUPRED3 vs VSL2B
accuracy_iupred3_vsl2b = accuracy_score(merged_cleaned_df['IUPRED3_Klasifikacija'], merged_cleaned_df['VSL2B_Klasifikacija'])
precision_iupred3_vsl2b = precision_score(merged_cleaned_df['IUPRED3_Klasifikacija'], merged_cleaned_df['VSL2B_Klasifikacija'])

# Ispis rezultata
print(f"Tačnost između ISUNSTRUCT i IUPRED3: {accuracy_isunstruct_iupred3:.2f}")
print(f"Preciznost između ISUNSTRUCT i IUPRED3: {precision_isunstruct_iupred3:.2f}")

print(f"Tačnost između ISUNSTRUCT i VSL2B: {accuracy_isunstruct_vsl2b:.2f}")
print(f"Preciznost između ISUNSTRUCT i VSL2B: {precision_isunstruct_vsl2b:.2f}")

print(f"Tačnost između IUPRED3 i VSL2B: {accuracy_iupred3_vsl2b:.2f}")
print(f"Preciznost između IUPRED3 i VSL2B: {precision_iupred3_vsl2b:.2f}")


Tačnost između ISUNSTRUCT i IUPRED3: 0.90
Preciznost između ISUNSTRUCT i IUPRED3: 0.91
Tačnost između ISUNSTRUCT i VSL2B: 0.90
Preciznost između ISUNSTRUCT i VSL2B: 0.97
Tačnost između IUPRED3 i VSL2B: 0.85
Preciznost između IUPRED3 i VSL2B: 0.98


Ovi rezultati ukazuju da svi alati uglavnom daju slične rezultate u pogledu klasifikacije "uređenih" i "neuređenih" regiona, ali ISUNSTRUCT i VSL2B se izdvajaju po visokoj preciznosti u prepoznavanju "uređenih" regiona

Na osnovu analize konzistentnosti, tacnosti, preciznosti i Kappa koeficijenta mozemo doneti sledece zakljucke:
-za formiranje modela uzimamo u obzir alate koji imaju vece vrednosti Kappa koeficijenta a to su ISUNSTRUCT I VSL2B jer su bolje uskladjeni

-tacnost između alata je visoka (oko 90%) za vecinu kombinacija, sto znači da su predikcije između alata uglavnom iste

-preciznost je takođe visoka, posebno između ISUNSTRUCT i VSL2B (97%) i između IUPRED3 i VSL2B (98%) za "uređene" regione. Ovi visoki brojevi ukazuju na to da su alati vrlo precizni u prepoznavanju "uređenih" regiona

-zbog visoke konzistentnosti (oko 83%) u obzir se mogu uzeti svi alati

Dakle, mogu koristiti ujedinjenje predikcije sva 3 alata ili 2 najbolja alata ISUNSTRUCT I VSL2B